In [4]:
import json

import folium
import pandas as pd

In [5]:
geodata_url = '../NYC Taxi Zones.geojson'

with open(geodata_url) as json_file:
    data = json.load(json_file)

In [6]:
truedata_url = "../do_location_aggregated.csv"
true_data = pd.read_csv(truedata_url)
true_data.set_index(true_data["DOLocationID"], inplace=True)
true_data["DOLocationID"] = true_data["DOLocationID"].apply(str)

In [7]:
m = folium.Map(location=[40.730610, -73.935242], zoom_start=10)

In [8]:
# https://stackoverflow.com/questions/61345801/featuregroup-layer-control-in-folium-only-one-active-layer
# wird benoetigt, damit bei den choropleths die map weiterhin im hintergrund angezeigt wird
base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
base_map.add_to(m)

In [9]:
fg_pickups = folium.FeatureGroup(name='Anzahl Ausstiege', overlay=False).add_to(m)
fg_time = folium.FeatureGroup(name='Durchschnittliche Fahrzeit', overlay=False).add_to(m)
fg_distance = folium.FeatureGroup(name='Durchschnittliche Fahrstrecke', overlay=False).add_to(m)

In [10]:
bins_pickups = list(true_data['dropoff_count'].quantile([0, 0.1, 0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.95, 1]))
colormap_pickups = folium.Choropleth(
    geo_data=data,
    name="choropleth",
    data=true_data,
    bins=bins_pickups,  # mehr als neun bins können nicht mit der farbe dargestellt werden
    columns=['DOLocationID', 'dropoff_count'],
    key_on="feature.properties.location_id",
    fill_color="YlGnBu",  # YlGn YlGnBu
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Anzahl der Ausstiege",
    nan_fill_color="White",
    highlight=True
).geojson.add_to(fg_pickups)

for borough in colormap_pickups.data.get('features'):
    borough_id = borough.get('properties').get('location_id')
    try:
        avg_pickups = int(true_data.loc[int(borough_id)]['dropoff_count'])
        borough.get('properties')['dropoff_count'] = f"{avg_pickups:,}"
    except ValueError:
        borough.get('properties')['dropoff_count'] = None
    except KeyError:
        borough.get('properties')['dropoff_count'] = None

folium.GeoJsonTooltip(["zone", "borough", "dropoff_count"], aliases=["Zone", "Borough", "Anzahl Ausstiege"]).add_to(
    colormap_pickups)

In [11]:
bins_time = list(true_data['minutes_average'].quantile([0, 0.1, 0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.95, 1]))
colormap_time = folium.Choropleth(
    geo_data=data,
    name="choropleth",
    data=true_data,
    bins=bins_time,  # mehr als neun bins können nicht mit der farbe dargestellt werden
    columns=['DOLocationID', 'minutes_average'],
    key_on="feature.properties.location_id",
    fill_color="YlGnBu",  # YlGn YlGnBu
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Dauer der Fahrt",
    nan_fill_color="White",
    highlight=True
).geojson.add_to(fg_time)

for borough in colormap_pickups.data.get('features'):
    borough_id = borough.get('properties').get('location_id')
    try:
        avg_times = int(true_data.loc[int(borough_id)]['minutes_average'])
        borough.get('properties')['minutes_average'] = f"{avg_times:,} min"
    except ValueError:
        borough.get('properties')['minutes_average'] = None
    except KeyError:
        borough.get('properties')['minutes_average'] = None

folium.GeoJsonTooltip(["zone", "borough", "minutes_average"],
                      aliases=["Zone", "Borough", "Dauer der Fahrten"]).add_to(
    colormap_time)

In [12]:
bins_distance = list(true_data['trip_distance_average'].quantile([0, 0.1, 0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.95, 1]))
colormap_distance = folium.Choropleth(
    geo_data=data,
    name="choropleth",
    data=true_data,
    bins=bins_distance,  # mehr als neun bins können nicht mit der farbe dargestellt werden
    columns=['DOLocationID', 'trip_distance_average'],
    key_on="feature.properties.location_id",
    fill_color="YlGnBu",  # YlGn YlGnBu
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Länge der Fahrt",
    nan_fill_color="White",
    highlight=True
).geojson.add_to(fg_distance)

for borough in colormap_pickups.data.get('features'):
    borough_id = borough.get('properties').get('location_id')
    try:
        avg_trip_distance = int(true_data.loc[int(borough_id)]['trip_distance_average'])
        borough.get('properties')['trip_distance_average'] = f"{avg_trip_distance:,} Meilen"
    except ValueError:
        borough.get('properties')['trip_distance_average'] = None
    except KeyError:
        borough.get('properties')['trip_distance_average'] = None

folium.GeoJsonTooltip(["zone", "borough", "trip_distance_average"],
                      aliases=["Zone", "Borough", "Länge der Fahrten"]).add_to(
    colormap_distance)

In [13]:
folium.TileLayer('cartodbpositron', overlay=True, name="View in Light Mode").add_to(m)
folium.LayerControl().add_to(m)

In [ ]:
m

In [15]:
m.save("../visuals/choropleth/choropleth-dropoff.html")